In [1]:
import pandas as pd
import matplotlib.pyplot  as plt
import numpy as np

%matplotlib inline

from collections import Counter

In [4]:
import glob
glob.glob("../Prepped data/17. CRS Service Users 2014/*")

['../Prepped data/17. CRS Service Users 2014/All service users.csv',
 '../Prepped data/17. CRS Service Users 2014/Carer age.csv',
 '../Prepped data/17. CRS Service Users 2014/Carer Ethnicity.csv',
 '../Prepped data/17. CRS Service Users 2014/Carer Gender.csv',
 '../Prepped data/17. CRS Service Users 2014/Carer Language.csv',
 '../Prepped data/17. CRS Service Users 2014/Carer Partnership.csv']

In [5]:
# Read in all the raw survey files
datasets = {fin: pd.read_csv("../Prepped data/" +fin) 
            for fin in glob.glob("../Prepped data/17. CRS Service Users 2014/*")}
    

## Load the data from Data set 8

Conver the really wide columns into a narrow set with sex / age/  ethnicity

In [6]:
data8 = pd.read_csv("../Prepped data/08.1 Ethnicity Sex Age England & Wales/8. Ethnicity Sex Age EW.csv")
#data8.head()

Get all the columns and look for the pivot and value columns

In [7]:
d8c = [c for c in data8.columns]
print(len(d8c), "columns in total")
d8c[:6]

366 columns in total


['LAD12CD',
 'Local.Authority.District.12',
 'Counties.&.UAs.code',
 'Counties.&.Unitary.Authorities',
 'Locality.cost.centre',
 'Locality']

Melt the dataset to narrow format 
Then add columns for sex / age / ethnicity 

In [8]:
data8melt = pd.melt(data8, id_vars=d8c[:6], value_vars=d8c[6:])

In [9]:
print(data8.shape)
print(data8melt.shape)

(348, 366)
(125280, 8)


In [10]:
def parse_col_name(cname):
    """Split a column into sex, age and ethnicity fields. 
    
    >>> parse_col_name('Sex:.All.persons;.Age:.All.categories:.Age;.Ethnic.Group:.White:.English/Welsh/Scottish/Northern.Irish/British')
    ('All.persons',
     'All.categories:.Age',
     'White:.English/Welsh/Scottish/Northern.Irish/British')
    
    """
    # All multi fields have sex / age / erthnicity
    fields = cname.split(";")
    sex_, age_, ethnic_ = fields
    sex = sex_.split(":", 1)[1]
    age = age_.split(":", 1)[1]
    ethnic = ethnic_.split(":", 1)[1]
    # Remove leading '.' on all columns
    return sex[1:], age[1:], ethnic[1:]


sexes, ages, ethnics = [], [], []
for col in d8c:
    # Ignore all the columns used for pivoting
    if col.startswith("Sex"):
        s, a, e = parse_col_name(col)
        sexes.append(s)
        ages.append(a)
        ethnics.append(e)

# Run to show counts of what is in each
c = Counter()
c.update(ages) # sexes / ethnics
c.most_common()

[('Age.65.and.over', 72),
 ('All.categories:.Age', 72),
 ('Age.0.to.24', 72),
 ('Age.25.to.49', 72),
 ('Age.50.to.64', 72)]

The long, ugly column names in the wide table are now a single column

Build a DataFrame with columns ["long name", "sex", "age", "ethnicity"] and then join


In [11]:
## Columns Lookup
column_lookup = pd.DataFrame({"colname": [c for c in d8c if c.startswith("Sex")], 
                              "sex": sexes, 
                              "age": ages, 
                              "ethnic": ethnics})



data8_sex_age_eth = pd.merge(data8melt, column_lookup, left_on="variable", right_on="colname")

print(data8melt.shape)
print(data8_sex_age_eth.shape)

print([c for c in data8_sex_age_eth.columns])
cols_of_interest = ['LAD12CD', 'Local.Authority.District.12', 'Counties.&.UAs.code', 
                    'Counties.&.Unitary.Authorities', 'Locality.cost.centre', 'Locality', 'variable', 'value', 
                    'age', 'ethnic', 'sex']

# Filter out columns and save to file
data8_sex_age_eth_final = data8_sex_age_eth[cols_of_interest]
data8_sex_age_eth_final.to_csv("../workingdata/data8_with_age_sex_ethnic.csv", index=False, sep=",")

(125280, 8)
(125280, 12)
['LAD12CD', 'Local.Authority.District.12', 'Counties.&.UAs.code', 'Counties.&.Unitary.Authorities', 'Locality.cost.centre', 'Locality', 'variable', 'value', 'age', 'colname', 'ethnic', 'sex']


In [12]:
# data8melt.head()

## Users of the Service Survey

In [ ]:
#
# Load all the data and turn into narrow format
# 

## Working directory is
#
# /+
#  /notebooks/<this file>
#  |           
#  /Prepped data/
#               |
#               /<survey 17>/
#                           | 
#                           + <data.csv>
#
# There is one CSV per survey
#
datasets = {fin: pd.read_csv("../Prepped data/" +fin) 
            for fin in glob.glob("../Prepped data/17. CRS Service Users 2014/*")}


# Locality needs to be based on the "Locality CRS 2014" columns
# This needs to be split according to the ratio in locations-remap.csv (in version control)
def single_narrow_dataset(datasets, locality_field = "Locality.CRS.2014"):
    
    # Read in a list of files from a directory and create a single DataFrame
    df_list = []
    
    for key, df in datasets.items():
        
        # Convert non-locality columns in a single column
        df_ = pd.melt(df, id_vars=locality_field, 
                          value_vars=[c for c in df.columns[2:]], 
                          value_name="value")
        # Select the columns we are interested in
        df_.columns = ["Location", "Variable", "Count"]
        # Create a dataset name from the original CSV
        dimname = os.path.basename(key)[:-4]
        df_["Dimension"] = dimname
        df_[["Location", "Dimension", "Variable", "Count"]]
        df_list.append(df_)
    
    # Stack all the columns 
    clean_df = pd.concat(df_list)
    # Get rid of all the non-total fields
    full_data = clean_df[clean_df.Variable != "Grand.Total"]
    
    # Re-map Avon, Gloucestershire & Wiltshire 
    # Banes, Swindon, Wiltshire and Care Services  0.274
    # and
    # Bristol, Gloucestershire and South Gloucestershire 72.6 
    LocationLookup = pd.read_csv("../workingdata/locations-remap.csv")
    full_data = pd.merge(full_data, LocationLookup, on="Location")
    
    # Set all the Nans to 0
    full_data["FullCount"] = full_data["Count"].fillna(0)
    
    # Aggregate by Location and Variable
    full_data = (full_data
                 .groupby(["NewLocation", "Variable", "Dimension"])
                ["FullCount"]
                .sum()
                .reset_index())
    
    # Get a within segment proportion
    full_data["FullProp"] = (full_data
                             .groupby(["Dimension", "NewLocation"])
                             ["FullCount"]
                             .transform(lambda x: x/x.sum()))
    return full_data[[]]



In [ ]:
full_data = single_narrow_dataset(datasets)
full_data.to_csv("../workingdata/known_data_by_question_response2.csv", index=False)